In [1]:
import argparse
import pickle
from pathlib import Path
import sys
import wandb

In [2]:
PROJECT_PATH = "/ziob/klaudia/FlowHMM"
sys.path.insert(1, PROJECT_PATH)

In [3]:
from clickstream_experiment.source.tagnn.models.tagnn.model import SessionGraph
from clickstream_experiment.source.tagnn.models.tagnn.wrappers import Trainer
from clickstream_experiment.source.tagnn.models.tagnn.utils import (
    Data, trans_to_cuda, find_edge_weights, read_data, calculate_embeddings, calculate_abx
)

ModuleNotFoundError: No module named 'src'